# Neeps - Hard

In [1]:
import getpass
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
pwd = getpass.getpass()
engine = create_engine(
    'postgresql+psycopg2://postgres:%s@192.168.31.31:15432/sqlzoo' % (pwd))
pd.set_option('display.max_rows', 60)


········


In [2]:
import datetime

In [3]:
ut_staff = pd.read_sql_table('ut_staff', engine)
ut_student = pd.read_sql_table('ut_student', engine)
ut_event = pd.read_sql_table('ut_event', engine)
ut_room = pd.read_sql_table('ut_room', engine)
ut_attends = pd.read_sql_table('ut_attends', engine)
ut_teaches = pd.read_sql_table('ut_teaches', engine)
ut_occurs = pd.read_sql_table('ut_occurs', engine)
ut_modle = pd.read_sql_table('ut_modle', engine)
ut_week = pd.read_sql_table('ut_week', engine)


## 11.
**co.CHt is to be given all the teaching that co.ACg currently does. Identify those events which will clash.**

In [4]:
t = (ut_event.rename(columns={'id': 'evt_id'})
     .merge(ut_teaches, left_on='evt_id', right_on='event')
     .merge(ut_occurs, left_on='evt_id', right_on='event'))
t['begin'] = pd.to_datetime(t['tod'], format='%H:%M')
t['end'] = t['begin'] + pd.to_timedelta(t['duration'], unit='hours')

a = (t.loc[t['staff']=='co.CHt'].merge(t.loc[t['staff']=='co.ACg'], 
                                       on=['week', 'dow']))
(a.loc[((a['begin_x']>=a['begin_y']) & (a['begin_x']<a['end_y'])) |
       ((a['begin_y']>=a['begin_x']) & (a['begin_y']<a['end_x'])),
      ['evt_id_x', 'evt_id_y']].drop_duplicates()
 .rename(columns={'evt_id_x': 'co.CHt', 'evt_id_y': 'co.ACg'}))

,co.CHt,co.ACg
0,co12005.T03,co12005.T01
2,co12005.T03,co72013.L02
3,co12005.T03,co72013.T03


## 12.
**Produce a table showing the utilisation rate and the occupancy level for all rooms with a capacity more than 60.**

> I don't know how 'utilisation rate' and 'occupancy level' are defined.

In [5]:
# theoretical full utilisation hours: 15 weeks * 5 days * 12 hours
close_time = (max(pd.to_datetime(ut_event['tod'], format='%H:%M') + 
                  pd.to_timedelta(ut_event['duration'], unit='hours')))
open_time = min(pd.to_datetime(ut_event['tod'], format='%H:%M'))
max_hrs = (close_time - open_time).seconds // 3600
t = ut_week.assign(max_hrs=5 * max_hrs)

# attended students per event
s = (ut_event.rename(columns={'id': 'event'})
     .merge(ut_occurs, on='event')
     .merge(ut_attends, on='event')
     .merge(ut_student, left_on='student', right_on='id')
     .merge(ut_room, left_on='room', right_on='id'))
s = s.groupby('room')[['sze', 'capacity']].sum().reset_index()

a = (ut_event.rename(columns={'id': 'event'}).merge(ut_occurs, on='event')
     .merge(ut_room.loc[ut_room['capacity']>60], left_on='room', right_on='id')
     .groupby('room')['duration']
     .sum().reset_index()
     .merge(s, on='room'))
a['util'] = a['duration'] / sum(t['max_hrs'])
a['occup_level'] = a['sze'] / a['capacity']
a[['room', 'util', 'occup_level']]

,room,util,occup_level
0,cr.SMH,0.24,0.274574


## 13.
**A one hour staff meeting is to be held between 09:00 and 17:00. Events which clash are to be cancelled. Identify the hour which will result in the least disruption.**

In [6]:
t = (ut_event.assign(tod=pd.to_datetime(ut_event['tod'], format='%H:%M'),
                     duration=ut_event['duration'].astype(int))
     .merge(ut_occurs, left_on='id', right_on='event')
     [['event', 'week', 'dow', 'tod', 'duration']])
for i in range(2, 10):
    t_ = t.loc[t['duration']==i]
    if t_.shape[0] > 0:
        t_.loc[:, 'tod'] = t_.loc[:, 'tod'] + pd.to_timedelta(i, unit='hours')
        t = pd.concat([t, t_])
        
(t.assign(weekday=t['dow'].replace({'Monday': 0, 'Tuesday': 1, 'Wednesday': 2,
                                    'Thursday': 3, 'Friday': 4}),
          hour=t['tod'].dt.hour)
 .groupby(['week', 'weekday', 'hour'])['event']
 .count().reset_index()
 .pivot(index=['weekday', 'hour'], columns='week', values='event'))

week          01  02  03  04  05  06  07  08  10  11  12  13
weekday hour                                                
0       9      4   4   4   4   4   4   4   4   4   4   4   4
        10     1   1   1   1   1   1   1   1   1   1   1   1
        11     8   8   8   8   8   8   8   8   8   8   8   8
        12     3   3   3   3   3   3   3   3   3   3   3   3
        13    10  10  10  10  10  10  10  10  10  10  10  10
        14     5   5   5   5   5   5   5   5   5   5   5   5
        15    10  10  10  10  10  10  10  10  10  10  10  10
        16     5   5   5   5   5   5   5   5   5   5   5   5
        17     8   8   8   8   8   8   8   8   8   8   8   8
        18     1   1   1   1   1   1   1   1   1   1   1   1
1       9      6   6   6   6   6   6   6   6   6   6   6   6
        10     7   7   7   7   7   7   7   7   7   7   7   7
        11    10  10  10  10  10  10  10  10  10  10  10  10
        12    10  10  10  10  10  10  10  10  10  10  10  10
        13    11  11  11  11  11  11  11  11  11  11  11  11
        14     5   5   5   5   5   5   5   5   5   5   5   5
        15    10  10  10  10  10  10  10  10  10  10  10  10
        16     8   8   8   8   8   8   8   8   8   8   8   8
        17     4   4   4   4   4   4   4   4   4   4   4   4
        18     4   4   4   4   4   4   4   4   4   4   4   4
        20     4   4   4   4   4   4   4   4   4   4   4   4
2       9      8   8   8   8   8   8   8   8   8   8   8   8
        10     3   3   3   3   3   3   3   3   3   3   3   3
        11     8   8   8   8   8   8   8   8   8   8   8   8
        12     7   7   7   7   7   7   7   7   7   7   7   7
        13     6   6   6   6   6   6   6   6   6   6   6   6
        14     6   6   6   6   6   6   6   6   6   6   6   6
        15     5   5   5   5   5   5   5   5   5   5   5   5
        16     7   7   7   7   7   7   7   7   7   7   7   7
        17     3   3   3   3   3   3   3   3   3   3   3   3
        18     2   2   2   2   2   2   2   2   2   2   2   2
3       9      4   4   4   4   4   4   4   4   4   4   4   4
        10     4   4   4   4   4   4   4   4   4   4   4   4
        11    11  11  11  11  11  11  11  11  11  11  11  11
        12     5   5   5   5   5   5   5   5   5   5   5   5
        13     9   9   9   9   9   9   9   9   9   9   9   9
        14     7   7   7   7   7   7   7   7   7   7   7   7
        15    11  11  11  11  11  11  11  11  10  10  10  10
        16     6   6   6   6   6   6   6   6   6   6   6   6
        17     8   8   8   8   8   8   8   8   7   7   7   7
        18     1   1   1   1   1   1   1   1   1   1   1   1
4       9      5   5   5   5   5   5   5   5   5   5   5   5
        10     5   5   5   5   5   5   5   5   5   5   5   5
        11     5   5   5   5   5   5   5   5   5   5   5   5
        12     5   5   5   5   5   5   5   5   5   5   5   5
        13     7   7   7   6   6   6   6   6   6   6   6   6
        14     3   3   3   3   3   3   3   3   4   4   4   4
        15     7   7   7   7   7   7   7   7   7   7   7   7
        16     4   4   4   4   3   3   3   3   4   4   4   4
        17     6   6   6   6   6   6   6   6   6   6   6   6

In [7]:
# You may find that week '09', '14', '15' are totally vacant
# or you can choose Mon 9am/12pm, Tue 5pm, Thu 9am, Fri 5pm

(t.assign(tod=t['tod'].dt.time)
 .loc[t['tod'].dt.hour.between(9, 17)]
 .groupby(['tod', 'dow'])['event']
 .count().reset_index().sort_values(['event', 'tod', 'dow'])
 .iloc[:1, :])


,tod,dow,event
6,10:00:00,Monday,12


## 14.
**Find all clashes - include the events which clash and the staff, student or rooms that they have in common.**

In [8]:
t = (ut_event.assign(start=pd.to_datetime(ut_event['tod'], format='%H:%M'))
    .rename(columns={'id': 'event'})
    .merge(ut_occurs, on='event'))

t['end'] = t['start'] + pd.to_timedelta(t['duration'], unit='hours')

a = t.merge(t, on=['week', 'dow', 'room']).query('event_x != event_y')

a = a.loc[(((a['start_x'] >= a['start_y']) & (a['start_x'] < a['end_y'])) |
           ((a['start_y'] >= a['start_x']) & (a['start_y'] < a['end_x']))) &
          (a['event_x'] > a['event_y']),
         ['event_x', 'event_y', 'room', 'week', 'dow']].drop_duplicates()
a

,event_x,event_y,room,week,dow
3304,co22020.T04,co22007.T01,co.LB47,01,Friday
3319,co22020.T04,co22007.T01,co.LB47,10,Friday
3328,co22020.T04,co22007.T01,co.LB47,11,Friday
3337,co22020.T04,co22007.T01,co.LB47,12,Friday
3346,co22020.T04,co22007.T01,co.LB47,13,Friday
3356,co22020.T04,co22007.T01,co.LB47,02,Friday
3372,co22020.T04,co22007.T01,co.LB47,03,Friday
3387,co22020.T04,co22007.T01,co.LB47,04,Friday
3396,co22020.T04,co22007.T01,co.LB47,05,Friday
3405,co22020.T04,co22007.T01,co.LB47,06,Friday


## 15.
**Produce a timetable for a group of full time students for week 1**

In [9]:
t = (ut_event.assign(tod=pd.to_datetime(ut_event['tod'], format='%H:%M'))
     .rename(columns={'id': 'event'})
     .merge(ut_occurs.loc[ut_occurs['week']=='01'], on='event')
     .merge(ut_attends, on='event'))
for i in range(2, 10):
    t_ = t.loc[t['duration']==i]
    if t_.shape[0] > 0:
        t_.loc[:, 'tod'] = t_.loc[:, 'tod'] + pd.to_timedelta(i, unit='hours')
        t = pd.concat([t, t_])

t['tod'] = t['tod'].dt.hour
t['flag'] = 'Y'
(t[['student', 'tod', 'dow', 'flag']]
 .sort_values(['dow', 'tod'])
 .drop_duplicates()
 .pivot(index='student', columns=['dow', 'tod'], values='flag')
 .fillna(''))

dow         Friday                         Monday  ... Wednesday              \
tod             9  10 11 12 13 14 15 16 17     9   ...        9  10 11 12 13   
student                                            ...                         
co.12008.Ea                                        ...                         
co.12008.Eb                     Y     Y            ...                         
co.12012.E                                         ...                  Y  Y   
co.22022.E       Y  Y     Y                        ...                         
co1.BAe                                         Y  ...         Y     Y         
...            ... .. .. .. .. .. .. .. ..    ...  ...       ... .. .. .. ..   
com.ST                                             ...                         
com.ST.a                                           ...                         
com.ST.b                                           ...                         
com.ST.th                                          ...                         
com.ST.tu                                          ...                         

dow                         
tod         14 15 16 17 18  
student                     
co.12008.Ea                 
co.12008.Eb                 
co.12012.E      Y           
co.22022.E                  
co1.BAe                     
...         .. .. .. .. ..  
com.ST       Y     Y        
com.ST.a                    
com.ST.b                    
com.ST.th                   
com.ST.tu                   

[86 rows x 50 columns]